In [2]:
import pandas as pd
import numpy as np
import time

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)

In [3]:
import cython
%load_ext cython
print(cython.__version__)

3.0.0


In [4]:
POSSIBLE_VALUES = [2,3,4,5,6,7,8,9,10,11]

DECK_COLUMNS = [str(i) for i in POSSIBLE_VALUES]
HAND_COLUMNS = ["score", "cards", "aces"]

In [5]:
GAME_COLUMNS = ["chance"] + ["player_"+colname for colname in HAND_COLUMNS] + ["dealer_"+colname for colname in HAND_COLUMNS] + DECK_COLUMNS

# SETUP

In [6]:
def get_deck(number_decks=6):
    """get a deck with the specified number of decks (default: 6)"""
    
    deck = pd.Series([number_decks*4]*len(DECK_COLUMNS) ,index=DECK_COLUMNS)
    deck.loc["10"] *= 4
    return deck

In [7]:
def get_game(deck=None, player_cards=[], dealer_cards=[], update_deck=True):
    """get a game with the specified deck, player cards and dealer cards"""

    if deck is None:
        deck = get_deck()
    
    game = pd.DataFrame(
        [1.0] + [0, 0, 0]*2 + deck.values.flatten().tolist(), 
        index=GAME_COLUMNS
    ).T.astype(int).astype({"chance":float})

    for player_card_value in player_cards:
        game = update_player_with_card(game, player_card_value, probabilistic=False, update_deck=update_deck)
    
    for dealer_card_value in dealer_cards:
        game = update_dealer_with_card(game, dealer_card_value, probabilistic=False, update_deck=update_deck)
    
    return game

In [8]:
def update_player_with_card(game, value, probabilistic=True, update_deck=True):
    """update the game with the player drawing a card with the specified value"""

    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["player_score"] += value
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game["player_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    return next_game

In [9]:
def update_dealer_with_card(game, value, probabilistic=True, update_deck=True):
    """update the game with the dealer drawing a card with the specified value"""

    assert np.all(game["dealer_score"] >= 0) and np.all(game["dealer_score"] < 17), f"there are finished hands, but dealer tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["dealer_score"] += value
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game["dealer_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -1
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1 

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

    return next_game

In [10]:
def order_game_columns(game):
    """order the columns of the game dataframe"""

    return game[GAME_COLUMNS + [colname for colname in game.columns if colname not in GAME_COLUMNS]]

# PLAYER

In [11]:
def player_hits_once(game, keep_drawn_column=False):
    """get updated game when player hits once, throws error for finished hands"""

    assert np.all(game["dealer_cards"] != 0), f"dealer must have one card showing. hands: {game}"    
    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to hit. hands:\n{game}"
    assert game[DECK_COLUMNS].sum(axis=1).max() == game[DECK_COLUMNS].sum(axis=1).min(), f"there are decks with different number of cards. hands:\n{game}"
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values

    # TODO: 0 boundary check, what if there are no cards of that type left?
    # currently works, because the chance is 0 anyways

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["player_score"] += next_game["drawn"]
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "player_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    # drop drawn column
    if not keep_drawn_column:
        next_game = next_game.drop(columns=["drawn"])

    # group and reorder
    next_game = next_game.groupby(next_game.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()
    next_game = order_game_columns(next_game)

    return next_game.sort_values("player_score")


In [12]:
def player_hits_once_where_possible(game, verbose=False):
    """get updated game when player hits once, finished hands are kept as-is"""

    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"  

    is_playable = (game["player_score"] >= 0) & (game["player_score"] != 22)
    if is_playable.sum() == 0:
        if verbose : print("All possible games are finished")
        return game
    elif is_playable.sum() > 0:
        if verbose : print(f"Only {is_playable.sum()} games are playable")
    
    final = game[~is_playable].copy()
    playable = game[is_playable].copy()

    next_possible = player_hits_once(playable)
    final = pd.concat((final, next_possible), axis=0, ignore_index=True)

    # group and reorder
    final = final.groupby(final.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()
    final = order_game_columns(final)

    return final.sort_values("player_score")

# DEALER

In [13]:
def dealer_get_score_chances_for_deck(game):
    """extends game by possible dealer outcomes"""
    
    assert set(DECK_COLUMNS).issubset(set(game.columns.tolist())), "Deck Info not in columns"
    assert set(["player_score","player_cards","player_aces","dealer_score","dealer_aces","dealer_cards","chance"]).issubset(game.columns.tolist()), "not all dealer info in columns"
    assert "drawn" not in game.columns, f"drawn column already exists. hands:\n{game}"

    # keep original deck info
    for deck_column in DECK_COLUMNS:
        game["orig_"+deck_column] = game[deck_column]

    is_playable = (game["dealer_score"] >= 0) & (game["dealer_score"] < 17)
    final = game[~is_playable]
    playable = game[is_playable]

    while len(playable) > 0:
        # add possible cards to draw
        next_game = pd.merge(playable, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

        # helper diag matrix
        diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(playable),1))

        # update drawing chance
        next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values
        
        # TODO: 0 boundary check, what if there are no cards of that type left?
        # currently works, because the chance is 0 anyways

        # update deck info
        next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

        # updating score
        next_game["dealer_score"] += next_game["drawn"]
        # updating cards
        next_game["dealer_cards"] += 1
        # updating aces
        next_game.loc[next_game["drawn"]== 11, "dealer_aces"] += 1

        # if necessary, using ace as 1 instead of 11
        acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
        next_game.loc[acenecessary,"dealer_score"] -= 10
        next_game.loc[acenecessary,"dealer_aces"] -= 1

        # set bust to -2
        next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1

        # set blackjack to 22
        next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

        # removing drawn column
        next_game = next_game.drop("drawn", axis="columns")
        
        cols = game.columns.tolist()
        cols.remove("chance")
        next_game = next_game.groupby(cols).agg({'chance': 'sum'}).reset_index()

        is_playable = (next_game["dealer_score"] < 17) & (next_game["dealer_score"] >= 0)
        # finished hands are taken out of the game
        final = pd.concat((final, next_game[~is_playable]), axis=0, ignore_index=True)
        # playable hands
        playable = next_game[is_playable].copy()

    grouped = final.groupby(["orig_"+col for col in DECK_COLUMNS] + ["player_score","player_cards","player_aces","dealer_score","dealer_aces","dealer_cards"]).agg({'chance':'sum'}).reset_index()
    
    return grouped

In [14]:
def dealer_draws_once(game):
    """[DEPRECATED] get updated game when dealer draws one card, throws error for finished hands"""

    assert np.all(game["dealer_score"] >= 0) and np.all(game["dealer_score"] < 17), f"there are finished hands, but dealer tried to draw. hands:\n{game}"
    assert "drawn" not in game.columns, f"drawn column already exists. hands:\n{game}"
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values
    
    # TODO: 0 boundary check, what if there are no cards of that type left?
    # currently works, because the chance is 0 anyways

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["dealer_score"] += next_game["drawn"]
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "dealer_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

    # removing drawn column
    next_game = next_game.drop("drawn", axis="columns")
    
    next_game = next_game.groupby(game.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()

    next_game = order_game_columns(next_game)

    return next_game.sort_values("dealer_score")

In [15]:
def dealer_plays(game):
    """[DEPRECATED] extends game rows by possible dealer outcomes
    use get_dealer_outcomes instead"""

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    is_playable = (game["dealer_score"] >= 0) & (game["dealer_score"] < 17)
    final = game[~is_playable]
    playable = game[is_playable]

    if len(playable) == 0:
        if verbose : print("Dealer has finished all hands")
        return game

    while len(playable) > 0:
        next_possible = dealer_draws_once(playable)

        is_playable = (next_possible["dealer_score"] < 17) & (next_possible["dealer_score"] >= 0)
        # finished hands are taken out of the game
        final = pd.concat((final, next_possible[~is_playable]), axis=0, ignore_index=True)
        # playable hands
        playable = next_possible[is_playable]
    
    return final

In [86]:
def get_dealer_outcomes(game, verbose=False):
    """extends game rows by possible dealer outcomes, adds column "expected_return" """

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"

    # get distinct decks
    distinct_decks = game.copy().groupby(["player_score","player_cards","player_aces","dealer_score","dealer_cards","dealer_aces"] + DECK_COLUMNS).agg({"chance":"min"}).reset_index()
    distinct_decks2 = game.copy().groupby(["player_score","player_cards","player_aces","dealer_score","dealer_cards","dealer_aces"] + DECK_COLUMNS).agg({"chance":"max"}).reset_index()

    assert np.all(np.isclose(distinct_decks, distinct_decks2)), "chances are not the same for same decks"

    if verbose:
        print("game\n", game)
        print(f"Simplified from {len(game)} to {len(distinct_decks)} games")
        print("distinct decks\n", distinct_decks)

    dealer_scores = dealer_get_score_chances_for_deck(distinct_decks)
    dealer_scores = get_return_column(dealer_scores)

    dealer_scores["expected_return_normed"] = dealer_scores["return"] * dealer_scores["chance"]
    
    if verbose:
        print("dealer scores\n", dealer_scores[dealer_scores["player_score"] == 20])

    dealer_scores_grouped = dealer_scores.groupby(["orig_"+c for c in DECK_COLUMNS]).agg({"expected_return_normed":"sum","chance":"sum"}).reset_index()

    if verbose:
        print("dealer scores grouped\n", dealer_scores_grouped)

    game = game.rename(columns={"chance":"chance_orig"})
    
    # merge back to original game
    game = pd.merge(game, dealer_scores_grouped.loc[:, ["orig_"+s for s in DECK_COLUMNS] + ["expected_return_normed","chance"]], left_on=DECK_COLUMNS, right_on=["orig_"+s for s in DECK_COLUMNS], how="left")

    assert np.all(np.isclose(game["chance_orig"],game["chance"]))
    game["expected_return"] = game["chance_orig"] * (game["expected_return_normed"] / game["chance"])

    return game


# RETURNS

In [87]:
def get_expected_return_standing_vec(game, verbose=False):
    """add column "expected return" to game,
    player has minimum of 2 cards, dealer has 1 upcard
    assumes perfect play"""

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"

    # bust hands
    is_bust = game["player_score"] == -2
    finished = game[is_bust].copy()
    playable = game[~is_bust].copy()

    if verbose:
        print("Calculating possible returns for each game...")
        print(f"Bust games: {len(finished) / len(game) * 100:.2f}%")

    dealer_outcomes = get_dealer_outcomes(playable, verbose)

    dealer_outcomes = dealer_outcomes.drop(columns=["chance_orig","expected_return_normed"])

    if verbose:
        print("dealer outcomes\n", dealer_outcomes[[col for col in dealer_outcomes.columns if ("orig_" not in col)]])

    finished["expected_return"] = -finished["chance"]
    
    final = pd.concat((finished, dealer_outcomes[[col for col in dealer_outcomes.columns if ("orig_" not in col)]]), axis=0, ignore_index=True)
    return final


In [88]:
def get_return_finished(game):
    """get the probabilities of each possible return of the current state for the specified game
    no further moves are made"""

    assert np.all((game["dealer_score"] < 0) | (game["dealer_score"] >= 17)), "dealer has not finished yet"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    # blackjack win
    blackjack_win = (game["player_score"] == 22) & (game["dealer_score"] != 22)
    blackjack_win_chance = game.loc[blackjack_win, "chance"].sum()

    # win
    win = (game["player_score"] > game["dealer_score"]) & (game["player_score"] != 22)
    win_chance = game.loc[win, "chance"].sum()

    # lose
    lose = (game["player_score"] < game["dealer_score"])
    lose_chance = game.loc[lose, "chance"].sum()

    # push
    push = (game["player_score"] == game["dealer_score"])
    push_chance = game.loc[push, "chance"].sum()

    expected_return = blackjack_win_chance*1.5 + win_chance*1 + push_chance*0 + lose_chance*(-1)

    return pd.Series({
        "blackjack_win_chance": blackjack_win_chance,
        "win_chance": win_chance,
        "push_chance": push_chance,
        "lose_chance": lose_chance,
        "expected_return": expected_return
    })

In [89]:
def get_return_standing(game, verbose=False):
    """get the expected return of standing (or doing nothing in case the player is already finished)"""

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."

    game = dealer_plays(game.copy())

    if verbose : print(game)

    return get_return_finished(game)

In [90]:
def get_return_hitting_once(game, verbose=False):
    """get the expected return of hitting exactly once, throws error for finished hands"""
    
    assert np.all(game["player_score"] < 22), "player has a natural"
    assert np.all(game["player_score"] >= 0), "player is already busted"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."

    game = player_hits_once(game.copy())

    return get_return_standing(game, verbose=verbose)

In [91]:
def get_return_hitting_once_where_possible(game, verbose=False):
    """get the expected return of hitting exactly once, finished hands are kept as-is"""

    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"  
    
    game = player_hits_once_where_possible(game.copy())

    return get_return_standing(game, verbose=verbose)

In [92]:
def get_return_doubling_down(game, verbose=False):
    """get the expected return of doubling down, throws error for finished hands"""

    assert np.all(game["player_score"] < 22), "player has a natural"
    assert np.all(game["player_score"] >= 0), "player is already busted"
    assert np.all(game["player_cards"] == 2), "player does not have exactly 2 cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."

    game = player_hits_once(game.copy())

    return_chances = get_return_standing(game, verbose=verbose)
    return_chances["expected_return"] *= 2
    
    return return_chances

In [93]:
def get_return_doubling_down_where_possible(game, verbose=False):
    """get the expected return of doubling down, finished hands are kept as-is"""

    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"  
    
    is_playable = (game["player_score"] >= 0) & (game["player_score"] != 22)
    if is_playable.sum() == 0:
        print("All possible games are finished")
        return game
    elif is_playable.sum() > 0:
        print(f"Only {is_playable.sum()} games are playable")
    
    final = game[~is_playable].copy()
    playable = game[is_playable].copy()

    next_possible = player_hits_once(playable)
    final = pd.concat((final, next_possible), axis=0, ignore_index=True)

    return_chances = get_return_standing(final, verbose=verbose)
    return_chances["expected_return"] *= 2
    
    return return_chances

In [94]:
def get_return_column(game):
    """add "return" column to game dataframe, denoting the return of the game"""

    assert np.all((game["dealer_score"] < 0) | (game["dealer_score"] >= 17)), "dealer has not finished yet"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    game["return"] = pd.Series(np.zeros(len(game)))

    game.loc[(game["player_score"] == 22) & (game["dealer_score"] != 22), "return"] = 1.5
    game.loc[(game["player_score"] > game["dealer_score"]) & (game["player_score"] != 22), "return"] = 1
    game.loc[game["player_score"] == game["dealer_score"], "return"] = 0
    game.loc[game["player_score"] < game["dealer_score"], "return"] = -1

    return game

# TESTING

In [119]:
def get_best_return_updated(game, verbose=False):
    """add column "expected_return" to game dataframe
    assumes best play"""

    t1 = time.time()

    stand_return = get_expected_return_standing_vec(game)

    if np.all(game["player_score"] == 22):
        return pd.Series({
            "stand":stand_return["expected_return"].iloc[0], 
            "hit":-1,
            "double":-2,
            "surrender":-0.5,
            "best":"stand"
            })

    is_playable = (game["player_score"] >= 0) & (game["player_score"] != 22) & (game["player_score"] != 21)

    playable = game[is_playable].copy()
    final = game.copy()

    # print("calculating all possible moves...")
    while len(playable) > 0:
        next_game = player_hits_once(playable, keep_drawn_column=True)
        drawnsum = len([i for i in next_game.columns.tolist() if "drawn" in i])
        next_game = next_game.rename(columns={"drawn":"drawn_"+str(drawnsum)})
        final["drawn_"+str(drawnsum)] = np.zeros(len(final)).astype(int)
        
        final = pd.concat((final, next_game), axis=0, ignore_index=True)

        is_playable = (next_game["player_score"] >= 0) & (next_game["player_score"] != 22) & (next_game["player_score"] != 21)
        playable = next_game[is_playable].copy()

    # print("calculated all player moves", time.time() - t1)

    final = get_expected_return_standing_vec(final, verbose)

    drawn_columns = [i for i in final.columns.tolist() if "drawn" in i]

    # get double scores if player has 2 cards and can draw at least once 
    if np.all(game["player_cards"] == 2) and "drawn_1" in drawn_columns and "drawn_2" in drawn_columns:
        double_return = final.loc[(final["drawn_1"] != 0) & (final["drawn_2"] == 0), "expected_return"].sum() * 2
    elif np.all(game["player_cards"] == 2) and "drawn_1" in drawn_columns and "drawn_2" not in drawn_columns:
        double_return = final.loc[(final["drawn_1"] != 0), "expected_return"].sum() * 2
    else:
        double_return = None

    for i in range(len(drawn_columns)):
        col = drawn_columns[len(drawn_columns)-i-1]
        cols = drawn_columns[:len(drawn_columns)-i-1]

        # get all games that can stand or hit in that column
        if len(cols) != 0:
            useable = (final[col] != 0) | ((final[col] == 0) & (final[cols[-1]] != 0) & (final["player_score"] > 0))
        else:
            useable = (final[col] != 0) | ((final[col] == 0) & (final["player_score"] > 0))
        used = final[useable].copy()

        stand_column = used[col] == 0
        used["stand"] = stand_column
        # group all games that hit
        grouped = used.groupby(cols + ["stand"]).agg({'chance': 'sum', "expected_return": "sum"}).reset_index()
        
        # if there are no drawn columns left, the expected returns can be returned
        if len(cols) != 0:
            best_options = grouped.groupby(cols).agg({"expected_return": "max", "chance":"mean"}).reset_index()
        else:
            assert stand_return["expected_return"].iloc[0] == grouped.loc[grouped["stand"] == True, "expected_return"].iloc[0], f"stand return is not equal to expected return of stand: {stand_return['expected_return'].iloc[0]} != {grouped.loc[grouped['stand'] == True, 'expected_return'].iloc[0]}"
            return_series = pd.Series(
                {
                    "stand": grouped.loc[grouped["stand"] == True, "expected_return"].iloc[0],
                    "hit": grouped.loc[grouped["stand"] == False, "expected_return"].iloc[0],
                    "double": double_return,
                    "surrender":-0.5
                })
            return_series["best"] = return_series.idxmax()
            return return_series
        
        # drop drawn column from unused games
        final = final[~useable].copy().drop(columns=[col])

        # only keep best options of used games
        retdf = pd.DataFrame({
            "chance": best_options["chance"],
            "expected_return": best_options["expected_return"]
        })
        for c in cols:
            retdf[c] = best_options[c]
        

        final = pd.concat([final, retdf], axis=0, ignore_index=True)


In [116]:
game = get_game(get_deck(6), [5,5], [9], update_deck=False)
print(game)

game = get_best_return_updated(game, verbose=True)
game

   chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0     1.0            10             2            0             9             1            0  24  24  24  24  24  24  24  24  96  24
Calculating possible returns for each game...
Bust games: 80.00%
game
             chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4  ...  drawn_1  drawn_2  drawn_3  drawn_4  drawn_5  drawn_6  drawn_7  drawn_8  drawn_9  drawn_10
0     1.000000e+00            10             2            0             9             1            0  24  24  24  ...        0        0        0        0        0        0        0        0        0         0
1     7.692308e-02            12             3            0             9             1            0  23  24  24  ...        2        0        0        0        0        0        0        0        0         0
2     7.692308e-02            13      

stand    -0.542247
hit       0.115492
double     0.14321
best        double
dtype: object

In [120]:
# TODO: store all calculated dfs that would require heave computation in a dataframe. Before calculating a ER, look up in this dataframe. Also save and retrieve it for multiple sessions
# TODO: function that can calculate ALL possibilities, even from the start up (no dealer/player cards). Then there should be an overall ER for playing the game
# TODO: implement splitting

# OLD

In [ ]:
game = get_game(None, [6,6], [4], update_deck=True)
print(pd.DataFrame({
    "standing":get_return_standing(game), 
    "hitting_once":get_return_standing(player_hits_once_where_possible(game)), 
    "hitting_twice":get_return_standing(player_hits_once_where_possible(player_hits_once_where_possible(game))), 
    "hitting_thrice":get_return_standing(player_hits_once_where_possible(player_hits_once_where_possible(player_hits_once_where_possible(game)))),
    "hitting_four_times":get_return_standing(player_hits_once_where_possible(player_hits_once_where_possible(player_hits_once_where_possible(player_hits_once_where_possible(game))))),
    "doubling":get_return_doubling_down(game)
    })
)

                      standing  hitting_once  hitting_twice  hitting_thrice  hitting_four_times  doubling
blackjack_win_chance  0.000000      0.000000       0.000000        0.000000            0.000000  0.000000
win_chance            0.398873      0.372015       0.135492        0.027146            0.003322  0.372015
push_chance           0.000000      0.045911       0.020710        0.004208            0.000492  0.045911
lose_chance           0.601127      0.582074       0.843798        0.968645            0.996186  0.582074
expected_return      -0.202253     -0.210060      -0.708306       -0.941499           -0.992863 -0.420120
